In [1]:
import pandas as pd

# 1. CARGA DE DATOS (Solo tus CSVs)
df_matches = pd.read_csv('Laliga_matches_23_24.csv') # Esta tiene los partidos
df_xg_table = pd.read_csv('Laliga_table_xg_2023_24.csv') # Esta tiene las estadísticas avanzadas

# 2. GENERAR VOLUMEN (> 500 FILAS)
# Un partido tiene dos equipos. Al separar 'Home' y 'Away', 
# convertimos los 380 partidos en 760 registros (uno por equipo por partido).
home_side = df_matches[['Match ID', 'Home Team', 'Score']].rename(columns={'Home Team': 'team', 'Score': 'result'})
away_side = df_matches[['Match ID', 'Away Team', 'Score']].rename(columns={'Away Team': 'team', 'result': 'result'})

df_maestro = pd.concat([home_side, away_side], ignore_index=True)

# 3. UNIR CON LA TABLA DE ESTADÍSTICAS (Merge)
# Añadimos los xPoints, xG y Puntos reales a cada registro de partido
df_final = pd.merge(df_maestro, df_xg_table, left_on='team', right_on='name', how='left')

# 4. LIMPIEZA FINAL
# Eliminamos la columna repetida 'name' y ordenamos
df_final = df_final.drop(columns=['name'])

# --- COMPROBACIÓN ---
print(f"✅ Filas totales: {df_final.shape[0]} (Requisito >500 cumplido)")
print(f"✅ Columnas totales: {df_final.shape[1]} (Requisito >5 cumplido)")
print("\nPrimeras filas del dataset unificado:")
print(df_final.head())

✅ Filas totales: 760 (Requisito >500 cumplido)
✅ Columnas totales: 12 (Requisito >5 cumplido)

Primeras filas del dataset unificado:
   Match ID           team result Score  idx  played       xg  xgConceded  \
0   4205343        Almeria  0 _ 2   NaN   17      38  49.3728     62.0616   
1   4205347        Sevilla    1_2   NaN   16      38  45.1687     56.7460   
2   4205351  Real Sociedad    1_1   NaN    6      38  45.5000     42.0143   
3   4205348     Las Palmas    1_1   NaN   20      38  33.4065     57.4859   
4   4205344  Athletic Club  0 _ 2   NaN    5      38  53.2360     41.9482   

     xPoints  xgDiff  xgConcededDiff  pts  
0  43.179331 -6.3728         12.9384   21  
1  43.305692  2.8313         -2.7460   41  
2  54.506721  5.5000         -3.0143   60  
3  37.441523 -0.4065        -10.4859   40  
4  58.832578  7.7640         -4.9482   68  


In [4]:
import pandas as pd

# 1. Cargar tus datasets
# El que ya tiene 760 filas (partidos + xG)
# (Asegúrate de haber ejecutado antes el código que crea 'df_final')
df_valores = pd.read_csv('valores_mercado.csv')

# 2. Diccionario de mapeo (Muy importante)
# Esto corrige las diferencias de nombre entre Transfermarkt y tu dataset de partidos
mapeo_nombres = {
    'Real Madrid': 'Real Madrid',
    'FC Barcelona': 'Barcelona',
    'Atlético de Madrid': 'Atletico Madrid',
    'Girona FC': 'Girona',
    'Athletic Bilbao': 'Athletic Club',
    'Real Sociedad': 'Real Sociedad',
    'Villarreal CF': 'Villarreal',
    'Valencia CF': 'Valencia',
    'Sevilla FC': 'Sevilla',
    'Real Betis Balompié': 'Real Betis',
    'Celta de Vigo': 'Celta Vigo',
    'UD Almería': 'Almeria',
    'Getafe CF': 'Getafe',
    'RCD Mallorca': 'Mallorca',
    'CA Osasuna': 'Osasuna',
    'UD Las Palmas': 'Las Palmas',
    'Rayo Vallecano': 'Rayo Vallecano',
    'Deportivo Alavés': 'Alaves',
    'Cádiz CF': 'Cadiz',
    'Granada CF': 'Granada'
}

# Aplicamos el mapeo al CSV del scraping para que los nombres coincidan con 'df_final'
df_valores['team'] = df_valores['team'].replace(mapeo_nombres)

# 3. UNIÓN FINAL (Merge)
# Añadimos las columnas de valor, edad y plantilla a tus 760 filas originales
df_final_completo = pd.merge(df_final, df_valores, on='team', how='left')

# 4. COMPROBACIÓN
print(f"✅ Dataset final listo con {df_final_completo.shape[0]} filas.")
print("\nPrimeras filas del dataset unificado con datos económicos:")
print(df_final_completo[['team', 'result', 'xg', 'valor_total_m', 'edad_media']].head())

# (Opcional) Guardar el resultado para no perderlo
df_final_completo.to_csv('Laliga_dataset_final_EDA.csv', index=False)

✅ Dataset final listo con 760 filas.

Primeras filas del dataset unificado con datos económicos:
            team result       xg  valor_total_m  edad_media
0        Almeria  0 _ 2  49.3728          99.40        25.3
1        Sevilla    1_2  45.1687         228.75        26.7
2  Real Sociedad    1_1  45.5000         498.50        25.2
3     Las Palmas    1_1  33.4065         112.40        26.2
4  Athletic Club  0 _ 2  53.2360         295.65        27.0
